# <center>CryptoCurrencies CA#1<center>
<h1><center>Introduction to Cryptography<center></h1>

#### <center>Mohammad Sadegh Aghili<center>
#### <center>810100274<center>

## StoryLine:
We have TinTin and Haddock on two seperate islands and they want to send messages to each other using morse codes!!    

However, there is a problem over there. Roberto Rastapopoulos (Their enemy!) is listening to their conversation to figure out what they are saying to each other!    

Now TinTin wants to use some encryption and decryption to ensure that Roberto couldn't understand what they are saying. Help TinTin to acheive his goal!

## Part 1: Symmetric Cryptography



**Important Note**: In all steps, use **AES-128** encryption method for encryption/decryption. You can use python cryptography libraries for this purpose.

TinTin decided to design a symmetirc encryption scheme for his connection with haddock. He asked you to design it for him using these steps:

In [192]:
import os
import rsa
import string
import random
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.hashes import SHA256
from cryptography.hazmat.primitives.asymmetric import padding

In [2]:
def symmetric_encryptor(input_message: str, key: str) -> bytes:
    fernet_key = Fernet(key)

    cipher_text = fernet_key.encrypt(input_message.encode())

    return cipher_text

In [16]:
def symmetric_decryptor(input_ciphertext: str, key: bytes) -> str:
    fernet_key = Fernet(key)

    decrypted_message = fernet_key.decrypt(input_ciphertext.encode()).decode()

    return decrypted_message

In [28]:
def symmetric_verifier(plaintext:str, ciphertext:str, key:str):
    if plaintext == ciphertext:
        return True
    else:
        return False

Now test your functions by encrypting & decrypting the ((confidential-message.txt)) file.

In [5]:
random_key = Fernet.generate_key()

with open("Confidential-Message.txt", "r") as file:
    confidential_message = file.read()

encrypted_message = symmetric_encryptor(confidential_message, random_key)

with open("Encrypted-Message.txt", "wb") as file:
    file.write(encrypted_message)

print("Confidential message encrypted and saved as Encrypted-Message.txt")

Confidential message encrypted and saved as Encrypted-Message.txt


In [19]:
with open("Encrypted-Message.txt", "r") as file:
    Encrypted_message = file.read()

decrypted_message = symmetric_decryptor(Encrypted_message, random_key)

with open("Decrypted-Message.txt", "w") as file:
    file.write(decrypted_message)

print("Confidential message encrypted and saved as Decrypted-Message.txt")

Confidential message encrypted and saved as Decrypted-Message.txt


In [29]:
if symmetric_verifier(confidential_message, decrypted_message, random_key):
    print("The original message and the decrypted message are the same.")
else:
    print("The original message and the decrypted message are different.")

The original message and the decrypted message are the same.


## Part 2: Asymmetric Cryptography

After TinTin found how to use symmetric encryption using a key, he faced another problem! How to share a key with haddock without letting Roberto finds it?        

Therefore, he decides to use an asymmetric crypto system.    
help him to acheive his goal by writing a program which:

In [139]:
def RSA_key_generator():
    (pubkey, privkey) = rsa.newkeys(512, poolsize=8)
    return privkey, pubkey

In [150]:
def asymmetric_encryptor(public_key_str: str, input_message: str) -> bytes:

    public_key = serialization.load_pem_public_key(public_key_str.encode(), backend=default_backend())

    encrypted_message = public_key.encrypt(
        input_message.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_message

In [135]:
def asymmetric_decryptor(private_key_str: str, ciphertext: bytes) -> str:
    
    private_key = serialization.load_pem_private_key(private_key_str.encode(), password=None, backend=default_backend())

    decrypted_message = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=SHA256()),
            algorithm=SHA256(),
            label=None
        )
    ).decode()
    return decrypted_message

In [136]:
def asymmetric_verifier(plaintext:str, ciphertext:str, public_key:str):
    if plaintext == ciphertext:
        return True
    else:
        return False

Now test your functions by encrypting & decrypting the ((confidential-message.txt)) file again using new asummetric functions.

In [172]:
private_key, public_key = RSA_key_generator()

# with open("private_key.pem", "rb") as private_key_file:
#     private_key_file.write(private_key)

# with open("public_key.pem", "wb") as public_key_file:
#     public_key_file.write(public_key)

print("RSA key pair generated and saved as private_key.pem and public_key.pem")

RSA key pair generated and saved as private_key.pem and public_key.pem


In [169]:
encoded_key = rsa.encrypt(random_key, public_key)
decoded_key = rsa.decrypt(encoded_key, private_key)

with open("encoded-key.txt", "wb") as private_key_file:
    private_key_file.write(encoded_key)

with open("decoded-key.txt", "wb") as public_key_file:
    public_key_file.write(encoded_key)

In [166]:
if symmetric_verifier(random_key, decoded_key, public_key):
    print("The original message and the decrypted message are the same.")
else:
    print("The original message and the decrypted message are different.")

The original message and the decrypted message are the same.


## Part 3: Digital signature

Write a function to sign a given message in the text file (Confidentail message) using private key which was made in the part 2.   
You have to sign the hash of the message, not the whole message (why?). (Use SHA-256 for hashing)

#### Signing the hash of the incoming message, rather than the entire message itself, is better way duo to: 
- <b>Efficiency:</b> Hash functions produce fixed-size outputs regardless of the size of the input data. Signing a hash is much faster than signing the entire message, especially for large messages.</br>
- <b>Consistency:</b> Hash functions produce deterministic outputs for the same input, ensuring that the same message always generates the same hash value. This consistency is crucial for verifying the signature.</br>
- <b>Security:</b> Cryptographic hash functions are designed to be collision-resistant, meaning it is computationally infeasible to find two different inputs that produce the same hash value. By signing the hash of the message, we ensure that any change to the message, no matter how small, will result in a completely different hash value. This provides integrity and ensures that the signed message cannot be modified without invalidating the signature.

In [179]:
def sign_message(private_key_str: str, input_message: str) -> bytes:
    message = input_message.encode()
    signature = rsa.sign(message, private_key, 'SHA-1')

    return signature

In [187]:
def message_verification(input_message: str, signature: bytes, public_key: str) -> bool:
    message = input_message.encode()
    if rsa.verify(message, signature, public_key):
        return True
    else:
        return False

In [189]:
signature = sign_message(private_key, confidential_message)
verifyer = message_verification(confidential_message, signature, public_key)
print(verifyer)

True


MAC (Message Authentication Code) and HMAC (Keyed-Hash Message Authentication Code) are cryptographic techniques used to ensure message integrity and authenticity in secure messaging.

- **MAC**: Short piece of information generated using a hash function and a secret key to authenticate a message. Used in protocols like TLS.

- **HMAC**: A specific type of MAC that incorporates a secret key into the calculation, providing stronger security guarantees.

**Uses**:
- Ensure message integrity and authenticity in secure messaging protocols like PGP and S/MIME.
- Protect confidentiality and integrity of messages exchanged between users in secure messaging applications.
- Detect and prevent message tampering, data corruption, and man-in-the-middle attacks.